# Question 1 - Sequence alignment

### Stav Cohen 316492776
### Ron Kozitsa 312544240

In [13]:
from typing import Iterable, List, Tuple
import numpy as np

In [14]:
class EditWeights:
    """
    Abstract base class for classes providing edit weights for sequence alignment operations.
    """

    def pair_weight(self, first_obj, second_obj) -> float:
        """
        Get the weight for aligning the pair of given objects.

        @param first_obj The object from the first sequence.
        @param second_obj The object from the second sequence.
        @return The match weight if the two objects are equal,
                otherwise the substitution weight.
        """
        pass

    def insertion_weight(self, obj) -> float:
        """
        Get the insertion weight of the given object.

        @param obj The inserted object in the second sequence.
        @return The insertion weight.
        """
        pass

    def deletion_weight(self, obj) -> float:
        """
        Get the deletion weight of the given object.

        @param obj The deleted object in the first sequence.
        @return The deletion weight.
        """
        pass

In [15]:
class LevenshteinWeights(EditWeights):
    def pair_weight(self, first_obj, second_obj) -> float:
        if first_obj == second_obj:
            return 0
        else:
            return -1

    def insertion_weight(self, obj) -> float:
        return -1

    def deletion_weight(self, obj) -> float:
        return -1


In [16]:
class UniformWeights(EditWeights):
    def pair_weight(self, first_obj, second_obj) -> float:
        if first_obj == second_obj:
            return 2
        else:
            return -1
    
    def insertion_weight(self, obj) -> float:
        return -0.75

    def deletion_weight(self, obj) -> float:
        return -0.75

In [17]:
class NestedUniformWeights(EditWeights):
    def pair_weight(self, first_obj, second_obj) -> float:
        return align_sequences(first_obj, second_obj, UniformWeights())[0]
    
    def insertion_weight(self, obj) -> float:
        return align_sequences(obj, "", UniformWeights())[0]

    def deletion_weight(self, obj) -> float:
        return align_sequences(obj, "", UniformWeights())[0]

In [36]:
def align_sequences(first_seq: Iterable,
                    second_seq: Iterable,
                    weights: EditWeights, debug: bool = False) -> Tuple[float, List[Tuple]]:
    """
    Perform global alignment between the two given sequences and compute the score of the optimal alignment with respect
    to the given edit weights.

    If the lengths of the input sequences are m and n, this function takes O(mn) time and uses O(mn) memory.

    @param first_seq The first sequence of aligned objects.
    @param second_seq The second sequence of aligned objects.
    @param weights The edit weights.
    @return The computed score of the optimal global alignment.
    @return An output list of the aligned object pairs:
            The list contains pairs of the form (a_i, b_j) of objects from the first and the second input
            sequence, respectively. The pair (None, b_j) represents object insertion in the second
           sequence, and the pair (a_i, None) represents object deletion from the first sequence.
    """

    # Codes for the various edit operations:
    _OP_NULL = 0
    _OP_PAIR = 1
    _OP_INS = 2
    _OP_DEL = 3

    # Create the matrix of alignment scores and the matrix of back pointers.
    first_len = len(first_seq)
    second_len = len(second_seq)

    scores_mat = np.zeros((first_len + 1, second_len + 1))
    ops_mat = np.zeros((first_len + 1, second_len + 1), dtype=np.int8)

    # Set up the initial matrix entry.
    scores_mat[0, 0] = 0
    ops_mat[0, 0] = _OP_NULL

    # Fill the 0'th row, corresponding to the insertion of the objects in the second sequence.
    j = 1
    for second_obj in second_seq:
        # Sum up the insertion weight of the current object in the second sequence with the predecessor entry.
        ins_wgt = scores_mat[0, j - 1] + weights.insertion_weight(second_obj)

        scores_mat[0, j] = ins_wgt
        ops_mat[0, j] = _OP_INS

        j += 1

    # Fill the rest of the matrix: Go over all objects of the first sequence.
    i = 1
    for first_obj in first_seq:
        # The 0'th column corresponds to the deletion of the objects in the first sequence.
        del_wgt = scores_mat[i - 1, 0] + weights.deletion_weight(first_obj)

        scores_mat[i, 0] = del_wgt
        ops_mat[i, 0] = _OP_DEL

        # Fill the rest of the row: Go over all objects of the second sequence.
        j = 1
        for second_obj in second_seq:
            # Accumulate the match or substitution weight with the weight of the corresponding predecessor entry.
            max_wgt = scores_mat[i - 1, j - 1] + weights.pair_weight(first_obj, second_obj)
            best_op = _OP_PAIR

            # Accumulate the current insertion weight with the weight of the corresponding predecessor entry.
            ins_wgt = scores_mat[i, j - 1] + weights.insertion_weight(second_obj)
            if ins_wgt > max_wgt:
                max_wgt = ins_wgt
                best_op = _OP_INS

            # Accumulate the current deletion weight with the weight of the corresponding predecessor entry.
            del_wgt = scores_mat[i - 1, j] + weights.deletion_weight(first_obj)
            if del_wgt > max_wgt:
                max_wgt = del_wgt
                best_op = _OP_DEL

            # Store the selected maximum weight and its corresponding operation.
            scores_mat[i, j] = max_wgt
            ops_mat[i, j] = best_op

            # Proceed to the next object in the second sequence.
            j += 1

        # Proceed to the next object in the first sequence.
        i += 1

    # Start with an empty list of aligned pairs, and from the given row and column.
    aligned_pairs = []
    i = first_len
    j = second_len

    while i > 0 or j > 0:
        # Check the edit operation at the current entry and act accordingly.
        curr_op = ops_mat[i, j]

        if curr_op == _OP_PAIR:
            # Pair operation: go back to the previous row and the previous column.
            i -= 1
            j -= 1
            aligned_pairs.append((first_seq[i], second_seq[j]))

        elif curr_op == _OP_INS:
            # Insertion operation: go back to the previous column in the same row.
            j -= 1
            aligned_pairs.append((None, second_seq[j]))

        elif curr_op == _OP_DEL:
            # Deletion operation: go back to the previous row in the same column.
            i -= 1
            aligned_pairs.append((first_seq[i], None))

        else:
            break

    # As we traced the alignment backwards, we have to reverse the list of aligned pairs.
    aligned_pairs.reverse()

    if debug or isinstance(weights, NestedUniformWeights):
        print(scores_mat)

    # Return the global alignment score and the aligned pairs.
    return scores_mat[first_len, second_len], aligned_pairs

def print_difference(lst: List[Tuple]) -> List[Tuple]:
    list_differences = [(a, b) for (a, b) in lst if a != b]
    print(f"Total of {len(list_differences)} differences")
    for (a,b) in list_differences:
        if a is None:
            print(f"Insert *{b}* from the second string")
        elif b is None:
            print(f"Delete *{a}* from the first string")
        else:
            print(f"Replace *{a}* by *{b}*")

In [50]:
print("="*50)
print("Part1: Two string alignment")
print("="*50)
str1 = "compassion"
str2 = "comparison"
print("Comparing alignments of by levenshtein weights:")
print(f"str1: {str1}")
print(f"str2: {str2}")
print("="*30)
print("Scores matrix:")
align_score, aligned_pairs = align_sequences(str1, str2, LevenshteinWeights(), debug=True)
print("="*30)
print("Results:")
print("Alignment score: ", align_score)
print("Levenstein distance:", int(np.abs(align_score)))
print("="*30)
print("Differences:")
print_difference(aligned_pairs)

Part1: Two string alignment
Comparing alignments of by levenshtein weights:
str1: compassion
str2: comparison
Scores matrix:
[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10.]
 [ -1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9.]
 [ -2.  -1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.]
 [ -3.  -2.  -1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.]
 [ -4.  -3.  -2.  -1.   0.  -1.  -2.  -3.  -4.  -5.  -6.]
 [ -5.  -4.  -3.  -2.  -1.   0.  -1.  -2.  -3.  -4.  -5.]
 [ -6.  -5.  -4.  -3.  -2.  -1.  -1.  -2.  -2.  -3.  -4.]
 [ -7.  -6.  -5.  -4.  -3.  -2.  -2.  -2.  -2.  -3.  -4.]
 [ -8.  -7.  -6.  -5.  -4.  -3.  -3.  -2.  -3.  -3.  -4.]
 [ -9.  -8.  -7.  -6.  -5.  -4.  -4.  -3.  -3.  -3.  -4.]
 [-10.  -9.  -8.  -7.  -6.  -5.  -5.  -4.  -4.  -4.  -3.]]
Results:
Alignment score:  -3.0
Levenstein distance: 3
Differences:
Total of 3 differences
Replace *s* by *r*
Replace *s* by *i*
Replace *i* by *s*


In [51]:
print("="*50)
print("Part2: Two sequence alignment")
print("="*50)
seq1 = "friends romans countrymen lend me your ears i come to bury caesar not to praise him".split()
seq2 = "my friends romans country men land me your ears i come to bury caesar not to praise him".split()
print("Comparing alignments of by levenshtein weights:")
print(f"seq1: {seq1}")
print(f"seq2: {seq2}")
print("="*30)
print("Scores matrix:")
align_score, aligned_pairs = align_sequences(seq1, seq2, LevenshteinWeights(), debug=True)
print("="*30)
print("Results:")
print("Alignment score: ", align_score)
print("Levenstein distance:", int(np.abs(align_score)))
print("="*30)
print("Differences:")
print_difference(aligned_pairs)

Part2: Two sequence alignment
Comparing alignments of by levenshtein weights:
seq1: ['friends', 'romans', 'countrymen', 'lend', 'me', 'your', 'ears', 'i', 'come', 'to', 'bury', 'caesar', 'not', 'to', 'praise', 'him']
seq2: ['my', 'friends', 'romans', 'country', 'men', 'land', 'me', 'your', 'ears', 'i', 'come', 'to', 'bury', 'caesar', 'not', 'to', 'praise', 'him']
Scores matrix:
[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13.
  -14. -15. -16. -17. -18.]
 [ -1.  -1.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12.
  -13. -14. -15. -16. -17.]
 [ -2.  -2.  -2.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11.
  -12. -13. -14. -15. -16.]
 [ -3.  -3.  -3.  -2.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11.
  -12. -13. -14. -15. -16.]
 [ -4.  -4.  -4.  -3.  -3.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11.
  -12. -13. -14. -15. -16.]
 [ -5.  -5.  -5.  -4.  -4.  -4.  -4.  -4.  -5.  -6.  -7.  -8.  -9. -10.
  -11. -12. -13. -14. -15.]
 [ -6.  -6.  -6.  -

In [52]:
print("="*50)
print("Part3: Uniform weights")
print("="*50)
str1 = "compassion"
str2 = "comparison"
print("Comparing alignments of by uniform weights:")
print(f"str1: {str1}")
print(f"str2: {str2}")
print("="*30)
print("Scores matrix:")
align_score, aligned_pairs = align_sequences(str1, str2, UniformWeights(), debug=True)
print("="*30)
print("Results:")
print("Alignment score: ", align_score)
print("="*30)
print("Differences:")
print_difference(aligned_pairs)

Part3: Uniform weights
Comparing alignments of by uniform weights:
str1: compassion
str2: comparison
Scores matrix:
[[ 0.   -0.75 -1.5  -2.25 -3.   -3.75 -4.5  -5.25 -6.   -6.75 -7.5 ]
 [-0.75  2.    1.25  0.5  -0.25 -1.   -1.75 -2.5  -3.25 -4.   -4.75]
 [-1.5   1.25  4.    3.25  2.5   1.75  1.    0.25 -0.5  -1.25 -2.  ]
 [-2.25  0.5   3.25  6.    5.25  4.5   3.75  3.    2.25  1.5   0.75]
 [-3.   -0.25  2.5   5.25  8.    7.25  6.5   5.75  5.    4.25  3.5 ]
 [-3.75 -1.    1.75  4.5   7.25 10.    9.25  8.5   7.75  7.    6.25]
 [-4.5  -1.75  1.    3.75  6.5   9.25  9.    8.25 10.5   9.75  9.  ]
 [-5.25 -2.5   0.25  3.    5.75  8.5   8.25  8.   10.25  9.5   8.75]
 [-6.   -3.25 -0.5   2.25  5.    7.75  7.5  10.25  9.5   9.25  8.5 ]
 [-6.75 -4.   -1.25  1.5   4.25  7.    6.75  9.5   9.25 11.5  10.75]
 [-7.5  -4.75 -2.    0.75  3.5   6.25  6.    8.75  8.5  10.75 13.5 ]]
Results:
Alignment score:  13.5
Differences:
Total of 3 differences
Delete *s* from the first string
Replace *s* by *r*
Inse

In [53]:
print("="*50)
print("Part4: Nested uniform weights")
print("="*50)
seq1 = "friends romans countrymen lend me your ears i come to bury caesar not to praise him".split()
seq2 = "my friends romans country men land me your ears i come to bury caesar not to praise him".split()
print("Comparing alignments of by nested uniform weights:")
print(f"seq1: {seq1}")
print(f"seq2: {seq2}")
print("="*30)
print("Scores matrix:")
align_score, aligned_pairs = align_sequences(seq1, seq2, NestedUniformWeights())
print("="*30)
print("Results:")
print("Alignment score: ", align_score)
print("="*30)
print("Differences:")
print_difference(aligned_pairs)

Part4: Nested uniform weights
Comparing alignments of by nested uniform weights:
seq1: ['friends', 'romans', 'countrymen', 'lend', 'me', 'your', 'ears', 'i', 'come', 'to', 'bury', 'caesar', 'not', 'to', 'praise', 'him']
seq2: ['my', 'friends', 'romans', 'country', 'men', 'land', 'me', 'your', 'ears', 'i', 'come', 'to', 'bury', 'caesar', 'not', 'to', 'praise', 'him']
Scores matrix:
[[  0.    -1.5   -6.75 -11.25 -16.5  -18.75 -21.75 -23.25 -26.25 -29.25
  -30.   -33.   -34.5  -37.5  -42.   -44.25 -45.75 -50.25 -52.5 ]
 [ -5.25  -5.75  12.5    8.     2.75   0.5   -2.5   -4.    -7.   -10.
  -10.75 -13.75 -15.25 -18.25 -22.75 -25.   -26.5  -31.   -33.25]
 [ -9.75  -7.25   8.    24.5   19.25  17.    14.    12.5    9.5    6.5
    5.75   2.75   1.25  -1.75  -6.25  -8.5  -10.   -14.5  -16.75]
 [-17.25 -14.75   0.5   17.    36.25  34.    31.    29.5   26.5   23.5
   22.75  19.75  18.25  15.25  10.75   8.5    7.     2.5    0.25]
 [-20.25 -17.75  -2.5   14.    33.25  38.5   39.    37.5   34.5   31

In [58]:
print("="*50)
print("Comparison between section 4 and section 2:")
print("Both have the same number of differences needed (4 differences)")
print("However the alignment scorre of section 4 is much higher since we use\n uniform weights method which adds positive score for a complete matching")
print("While levenstein uses just neutral score for a complete matching (And negative for no matching)")
print("In addition, since we use nested uniform weights in section 4, the alignment score of each word is not static\n but it's the alignment score of it's characters comparison which increases the score.")

Comparison between section 4 and section 2:
Both have the same number of differences needed (4 differences)
However the alignment scorre of section 4 is much higher since we use
 uniform weights method which adds positive score for a complete matching
While levenstein uses just neutral score for a complete matching (And negative for no matching)
In addition, since we use nested uniform weights in section 4, the alignment score of each word is not static
 but it's the alignment score of it's characters comparison which increases the score.
